In [1]:
!pip install datasets evaluate scikit-learn transformers==4.26.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2

In [2]:
!git clone https://github.com/amark-23/Sentiment-Analysis-Models.git
%cd slp-labs-NLP/NLPlab_main

Cloning into 'slp-labs-NLP'...
remote: Enumerating objects: 376, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 376 (delta 9), reused 4 (delta 4), pack-reused 364 (from 2)
Receiving objects: 100% (376/376), 75.05 MiB | 12.19 MiB/s, done.
Resolving deltas: 100% (156/156), done.
Updating files: 100% (82/82), done.
/content/slp-labs-NLP/NLPlab_main


In [7]:
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"
import evaluate
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from utils.load_datasets import load_MR, load_Semeval2017A

# === SELECT DATASET ===
DATASET = 'MR'  # or 'Semeval2017A'

# === Define models per dataset ===
MODEL_LIST = {
    "MR": [
        'siebert/sentiment-roberta-large-english',
        #"bert-base-multilingual-cased"
        'distilbert-base-uncased-finetuned-sst-2-english',
        'AnkitAI/reviews-roberta-base-sentiment-analysis'
    ],
    "Semeval2017A": [
        'cardiffnlp/twitter-roberta-base-sentiment',
        'finiteautomata/bertweet-base-sentiment-analysis',
        #'j-hartmann/sentiment-roberta-large-english-3-classes'
    ]
}

# === Evaluation metric ===
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

def prepare_dataset(X, y):
    return Dataset.from_dict({'text': X, 'label': y})

# === Load dataset ===
if DATASET == "Semeval2017A":
    X_train, y_train, X_test, y_test = load_Semeval2017A()
elif DATASET == "MR":
    X_train, y_train, X_test, y_test = load_MR()
else:
    raise ValueError("Invalid dataset")

# === Encode labels ===
le = LabelEncoder()
le.fit(list(set(y_train)))
y_train_enc = le.transform(y_train)
y_test_enc = le.transform(y_test)
n_classes = len(le.classes_)

# === Prepare HuggingFace datasets ===
train_set = prepare_dataset(X_train, y_train_enc)
test_set = prepare_dataset(X_test, y_test_enc)

# === Loop over models ===
for PRETRAINED_MODEL in MODEL_LIST[DATASET]:
    print(f"\n Fine-tuning: {PRETRAINED_MODEL}")

    # Tokenizer & model
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(
        PRETRAINED_MODEL, num_labels=n_classes)

    # Tokenize
    tokenized_train = train_set.map(tokenize_function)
    tokenized_test = test_set.map(tokenize_function)


    # Small subset for quick training (just for testing in Colab)
    train_subset_size = min(2000, len(tokenized_train))
    eval_subset_size = min(2000, len(tokenized_test))
    small_train = tokenized_train.shuffle(seed=42).select(range(train_subset_size))
    small_eval = tokenized_test.shuffle(seed=42).select(range(eval_subset_size))


    # Training setup
    args = TrainingArguments(
        output_dir=f"{PRETRAINED_MODEL.replace('/', '_')}_output",
        evaluation_strategy="epoch",
        num_train_epochs=3,
        per_device_train_batch_size=2,
        logging_steps=5,
        save_strategy="no"
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=small_train, #tokenized_train
        eval_dataset=small_eval,   #tokenized_test
        compute_metrics=compute_metrics,
    )

    # Fine-tune
    trainer.train()

    # Evaluate
    print(" Evaluation:")
    trainer.evaluate()



 Fine-tuning: siebert/sentiment-roberta-large-english


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--siebert--sentiment-roberta-large-english/snapshots/74cea614e245b0832c770ec9aa51bd58df965b9c/config.json
Model config RobertaConfig {
  "_name_or_path": "siebert/sentiment-roberta-large-english",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "NEG

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_d

...

Map:   0%|          | 0/662 [00:00<?, ? examples/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_d

Epoch,Training Loss,Validation Loss,Accuracy
1,0.322400,0.696157,0.844411
2,0.608200,0.791288,0.871601
3,0.001500,0.790411,0.873112


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 662
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 662
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 662
  Batch size = 8



 Evaluation:
